In [5]:
# Dependencies
import os
from bs4 import BeautifulSoup as bs
import urllib.request
import requests
import pymongo

# NASA Mars News Scrape

In [25]:
# Import HTML Code
html = urllib.request.urlopen('https://mars.nasa.gov/news/').read()
soup = bs(html, 'html.parser')

In [23]:
# Print all title texts
news_title = soup.find_all('div', class_='content_title')
for title in news_title:
    print(title.text)



Steep Slopes on Mars Reveal Structure of Buried Ice




NASA Mars Rover Team's Tilted Winter Strategy Works




NASA Builds Its Next Mars Rover Mission




Recurring Martian Streaks: Flowing Sand, Not Water?




Martian Ridge Brings Out Rover's Color Talents




Next Mars Rover Will Have 23 'Eyes'




In [24]:
# Print all paragraph texts
news_p = soup.find_all('div', class_='rollover_description_inner')
for p in news_p:
    print(p.text)


Researchers using NASA's Mars Reconnaissance Orbiter have found eight sites where thick deposits of ice beneath Mars' surface are exposed.


NASA's senior Mars rover, Opportunity, has just passed the shortest-daylight weeks of the long Martian year with its solar panels in encouragingly clean condition for entering a potential dust-storm season in 2018.




In just a few years, NASA's next Mars rover mission will be flying to the Red Planet.


Seasonal dark streaks on Mars that previously were described as possible signs of flowing water have steep slopes that are better matches to dry flow processes...


On a part of "Vera Rubin Ridge" where rover-team researchers sought to determine whether dust coatings are hiding rocks' hematite content, the Mast Camera (Mastcam) on NASA's Curiosity Mars rover took this image of a rock surface that had been brushed with the rover's Dust Removal Tool.


When NASA's Mars Pathfinder touched down in 1997, it had five cameras: two on a mast that popped

# JPL Mars Space Images - Featured Image

In [26]:
html = urllib.request.urlopen('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars').read()
soup = bs(html, 'html.parser')

In [71]:
# Print all title texts
featured_image = soup.find_all('div', class_='carousel_container')
for image in featured_image:
    print(url.get('href'))

NameError: name 'url' is not defined

In [43]:
featured_image

[<div class="carousel_container">
 <div class="carousel_items">
 <article alt="Titan's Northern Lakes: Salt Flats?" class="carousel_item" style="background-image: url('/spaceimages/images/wallpaper/PIA17470-1920x1200.jpg');">
 <div class="default floating_text_area ms-layer">
 <h2 class="category_title">
 </h2>
 <h2 class="brand_title">
 				  FEATURED IMAGE
 				</h2>
 <h1 class="media_feature_title">
 				  Titan's Northern Lakes: Salt Flats?				</h1>
 <div class="description">
 </div>
 <footer>
 <a class="button fancybox" data-description="NASA's Cassini spacecraft reveals the differences in the composition of surface materials around hydrocarbon lakes at Titan." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA17470_ip.jpg" data-link="/spaceimages/details.php?id=PIA17470" data-title="Titan's Northern Lakes: Salt Flats?" id="full_image">
 					FULL IMAGE
 				  </a>
 </footer>
 </div>
 <div class="gradient_container_top"></div>
 <div class="gradient

In [90]:
for url in soup.find_all('div', class_='carousel_container'):
    print(url.a)

<a class="button fancybox" data-description="NASA's Cassini spacecraft reveals the differences in the composition of surface materials around hydrocarbon lakes at Titan." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA17470_ip.jpg" data-link="/spaceimages/details.php?id=PIA17470" data-title="Titan's Northern Lakes: Salt Flats?" id="full_image">
					FULL IMAGE
				  </a>


In [ ]:
image = 